# Neural Network w/ backpropagation in Python from scratch

Lecture: https://www.youtube.com/watch?v=59Hbtz7XgjM
Post: https://cs231n.github.io/optimization-2/
Post: https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/
Video: https://www.youtube.com/watch?v=4shguqlkTDM
Code Inspiration: https://github.com/yacineMahdid/artificial-intelligence-and-machine-learning/blob/master/deep-learning-from-scratch-python/multi_layer_perceptron.ipynb (different data, try out different activations - sigmoid, ReLu, tanh)
Code Inspiration 2: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

### Functional Implementation

In [273]:
import numpy as np

In [274]:
# XOR
inp1 = [[0,0], [0,1], [1,0], [1,1]]
out1 = [0,1,1,0]

In [275]:
# 1. Initialize network with weights

data_len = len(inp1)
network = []
n_layers = 2
n_hidden = n_layers - 1
n_neurons = [2, 1] # layer 1, layer 2, ... , layer n
n_weights = [data_len, 1] # layer 1, layer 2, ... , layer n. Bias not included


def init_weights(n_weights):
    return np.random.rand(n_weights)

for layer in range(n_layers):
    l = [{'params': [np.random.rand() for n in range(n_weights[layer] + 1)]} for n in range(n_neurons[layer])]
    network.append(l)

i = 0
for layer in network:
    if i < n_hidden:
        layer.append({'type': 'hidden'})
    else:
        layer.append({'type': 'output'})
    i += 1

def print_layers(network):
    i = 1
    for layer in network:
        if layer[-1]['type'] == 'hidden':
            print(f'HIDDEN LAYER {i}')
        else:
            print('OUTPUT LAYER')
        print(layer)
        print(' ')
        i += 1

print_layers(network)

HIDDEN LAYER 1
[{'params': [0.0424649763541316, 0.7275132307972553, 0.2817894641574412, 0.0153838518846936, 0.92494042436315]}, {'params': [0.3038116314700229, 0.4938206162882226, 0.5733637314974005, 0.7677368136591562, 0.04837722371730124]}, {'type': 'hidden'}]
 
OUTPUT LAYER
[{'params': [0.40808710893356437, 0.2906956521598103]}, {'type': 'output'}]
 


In [276]:
# 2. Forward propagate

# Calculates the output of a single neuron -> (weights * inputs) + bias
def calc_neuron_output(params, inputs):
    bias = params[-1]
    output = bias
    for i in range(len(params) - 1):
        if any(isinstance(inp, list) for inp in inputs):
            for inp in inputs[i]:
                output += params[i] * inp
        else:
            output += params[i] * inputs[i]
    return output

# Activation functions
def sigmoid(output):
    return 1.0 / (1.0 + np.exp(-output))

def ReLu(output):
    return max(0, output)

def tanh(output):
    return (np.exp(output)-np.exp(-output))/(np.exp(output)+np.exp(-output))

activation_functions = ('sigmoid', 'ReLu', 'tanh')

def forward_propagate(inputs, activation_func):
    inp = inputs
    
    if activation_func.__name__ not in activation_functions:
        raise NameError('The activation function specified does not exist')
    
    for layer in network:
        outs = []
        for neuron in layer:
            if 'type' in neuron.keys():
                pass
            else:
                neuron_out = calc_neuron_output(neuron['params'], inp)
                neuron['output'] = neuron_out
                if layer[-1]['type'] == 'hidden':
                    neuron['activated'] = activation_func(neuron['output'])
                    outs.append(neuron['activated'])
                else:
                    neuron['activated'] = sigmoid(neuron['output']) # activation is output layer is sigmoid
        inp = outs
        
    return activation_func # to be used in backpropagation for the derivative

forward_propagate(inp1, ReLu)

print_layers(network)

HIDDEN LAYER 1
[{'params': [0.0424649763541316, 0.7275132307972553, 0.2817894641574412, 0.0153838518846936, 0.92494042436315], 'output': 1.9650108230872338, 'activated': 1.9650108230872338}, {'params': [0.3038116314700229, 0.4938206162882226, 0.5733637314974005, 0.7677368136591562, 0.04837722371730124], 'output': 2.651035198821237, 'activated': 2.651035198821237}, {'type': 'hidden'}]
 
OUTPUT LAYER
[{'params': [0.40808710893356437, 0.2906956521598103], 'output': 1.0925912379766434, 'activated': 0.7488693544926488}, {'type': 'output'}]
 


In [277]:
# 3. Back propagate error

# Derivatives of activation functions
def d_sigmoid(s):
    return s*(1-s)

def d_ReLu(r):
    return 1 if r > 0 else 0

def d_tanh(t):
    return 1-t**2

def backpropagate_neuron(network, i, expected_output):
    
    # Base case
    if network[i][-1]['type'] == 'output':
        value_to_pass = None
        
        for neuron in network[i]:
            if 'type' not in neuron.keys():
                actual_output = neuron['output']
                error = actual_output - expected_output
                value_to_pass = (actual_output, error)

        return value_to_pass
    # End of base case

    actual_output, error = backpropagate_neuron(network, i + 1, expected_output)
    
    for neuron in network[i]:
        if 'type' not in neuron.keys():
            #### HERE - DELTA CALCULATION
            neuron['delta'] = actual_output
    

backpropagate_neuron(network, 0, out1[1])

In [278]:
# 4. Train network
print_layers(network)

HIDDEN LAYER 1
[{'params': [0.0424649763541316, 0.7275132307972553, 0.2817894641574412, 0.0153838518846936, 0.92494042436315], 'output': 1.9650108230872338, 'activated': 1.9650108230872338, 'delta': 1.0925912379766434}, {'params': [0.3038116314700229, 0.4938206162882226, 0.5733637314974005, 0.7677368136591562, 0.04837722371730124], 'output': 2.651035198821237, 'activated': 2.651035198821237, 'delta': 1.0925912379766434}, {'type': 'hidden'}]
 
OUTPUT LAYER
[{'params': [0.40808710893356437, 0.2906956521598103], 'output': 1.0925912379766434, 'activated': 0.7488693544926488}, {'type': 'output'}]
 


In [ ]:
# 5. Predict on small dataset

In [ ]:
# 6. Predict on real world dataset - https://www.kaggle.com/datasets/whenamancodes/fraud-detection?resource=download

### OOP Implementation